# Classification of Newsgroups by Text

## Newsgroups Data

We will take a look at some of the twenty newsgroups dataset, another common dataset for classification.

More specifically, it is a list of emails within 20 different newsgroups. We will just focus on a few categories in particular. 

There will most likely be some difficulties or misclassification between emails in `atheism` and `religion` as well as between `comp.graphics` and `space`. Let's see...

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# We will use four of the twenty newsgroups
categories = ['alt.atheism',
              'talk.religion.misc',
              'comp.graphics',
              'sci.space']

twenty_train_subset = fetch_20newsgroups(subset='train', categories=categories)
twenty_test_subset = fetch_20newsgroups(subset='test', categories=categories)

Now we have lists of messages (as strings) in the `.data` members.

In [ ]:
twenty_train_subset.data[0:2]

In [ ]:
print twenty_train_subset.target[0:2]
print twenty_train_subset.target_names

## Count Vectorization

Recall how we can generate features from text using the CountVectorizer. We can "fit" a body of text using a count vectorizer, which will give the CountVectorizer a total vocabulary. When we call transform again using the fitted vectorizer, it will return a matrix (sparse matrix) of size `document x vocabulary`, where each element is the frequency of that word.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Turn the text documents into vectors of word frequencies
vectorizer = CountVectorizer()
vectorizer.fit(twenty_train_subset.data)

X_train = vectorizer.transform(twenty_train_subset.data)

Again, this makes a matrix of word counts, where each row is a document and each column is the word, the cell matrix[document, word] contains the count of word in document.

We can expand on this by setting the ngram_range. This parameter allows us to set each column not only as one word, but possibly sequences of words.

We can also provide an argument for `stopwords` that will strip out any english style stop words such as "and", "or", "other", etc for words that don't provide significance.

In [ ]:
# Include every 1-gram, 2-gram, and 3-gram
vectorizer = CountVectorizer(ngram_range=(1,3),stop_words='english')
vectorizer.fit(twenty_train_subset.data)
X_train = vectorizer.transform(twenty_train_subset.data)

Additionally, we could use a tf-idf representation, which stands for Term Frequency - Inverse Document Frequency.

This value is the product of two intermediate values, the Term Frequency and the Inverse Document Frequency.

The Term Frequency is equivalent to the CountVectorizer features, the number of times or count that a word appear in the document. This is our most basic representation of text.

To establish Inverse Document Frequency, first let's define Document Frequency. This is the percentage of documents that a particular word appears in. For example, the word `the` might appear in 100% of documents, while words like `Syria` would likely have low document frequency. Inverse Document Frequency is simply 1 / Document Frequency (although often the log is also taken).

So tf-idf is Term Frequency * Inverse Document Frequency, or similar to Term Frequency / Document Frequency. The intuition is that words that have high weight are those that appear a lot in this document and/or appear in very few other documents (somehow unique to this document).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()

# This is short hand for calling fit, and then transform
X_train = vectorizer.fit_transform(twenty_train_subset.data)

We can put this together with our other tricks as well.

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
X_train = vectorizer.fit_transform(twenty_train_subset.data)

For computational ease, let's just look at single word patterns

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(twenty_train_subset.data)

# Exercise - Naive Bayes vs. Decision Trees

We originally reviewed the CountVectorizer concept during our exploration of Naive Bayes. This time, we'll use a tf-idf vectorizer to create our X_train matrix, and compare the performance of Naive Bayes and Decision Trees.


1. Using Naive Bayes, what is your cross validation score on the training data (5 fold using StratifiedShuffleSplit) with a tf-idf matrix?
2. What is your score on the test set when you fit on the entire training set using Naive Bayes?
3. With a similar setup, how well do decision trees perform on training and test? Try comparing the performance of the trees with various max_depth.


**Notes:**
* Using MultinomialNB may provide the best results when creating a Naive Bayes model.
* Note, you will have to define y_train, X_test, and y_test variables.
* You will also have to `transform` the data into a tf-idf matrix for the test set (**Don't call fit again! Remember that `fit` is run on the training set to get a total vocabulary. Once we have this vocabulary, we run transform on new data to transform the known words**)
* Remember not to contaminate your training and test sets. Ideally, the test set should be looked at once at the end. Otherwise, you could end up fitting to the test set.


# Bagging (Bootstrap Aggregation)

Let's explore how bootstrap aggregation can give us stronger results.

Bagging tends to work well when the base classifiers (weak learners) have a low bias but high variance. By averaging the predictions over these base classifiers, we can reduce the overall variance of the final predictions.

In [ ]:
from sklearn.ensemble import BaggingClassifier

# Let's view the arguments in the model
# By default, DecisionTreeClassifier used, 
#  but can also use any base classifier
model_bag = BaggingClassifier()

In [ ]:
model_bag.fit(X_train,y_train)

In [ ]:
model_bag.score(X_train,y_train)

# Exercise - Bagging

* How does bagging compare to a base decision tree classifier with 1, 5, 10, 20, and 30 estimators on the training set? 
* How about on the testing set?
* Plot the improvements on both training and testing sets

If you finish early, feel free to build a bagging ensemble using a different base classifier, such as NaiveBayes or SVM. You may see that SVM does poorly with a default rbf kernel.

Bagging works very well when the base classifier has a low bias but a high variance.

------
# Random Forests

Random Forests are very popular ensemble classifiers. They are relatively simple to use (very few parameters to set and easy to avoid overfitting). 

The only parameter we are really worried about is the number of trees we want to create - n_estimators in sklearn.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Default # of vars is sqrt(features)
model_rf = RandomForestClassifier()

We can use predict using our 20-newsgroup dataset above

In [ ]:
model_rf.fit(X_train,y_train)

In [ ]:
model_rf.score(X_train,y_train)

We can access the results and parameters of each individual estimator in the random forest through `estimators_`

In [ ]:
model_rf.estimators_[0:2]

Random Forests can quantify the importance of features. Unlike with logistic regression, we don't have coeffcients that tell us relative impact. But we can keep track of what features give us the best splits.

In [ ]:
# This prints the top 10 most important features
print sorted(zip(model_rf.feature_importances_, 
                 vectorizer.get_feature_names()), 
             reverse=True)[:20]

# Exercise - Random Forest Estimator Size

* Compare training vs testing performance as you increase the number of estimators (i.e. from 1 - 100 for instance). Plotting will help visualize this.
* In addition, anecdotally compare the performance to that of bagged trees.

Extra: If you finish early, look at the top feature importances for a random forest. Is this similar to the feature importances coming out of a bagged model? Since the BaggingClassifier is generic and does not have a feature\_importances\_ attribute, you may need to roll your own baggingclassifier (i.e. generate your own function to bootstrap samples as well as create decision trees through a for loop, while accessing the feature\_importances\_ of each individual tree)

For ensemble methods, we would like our base classifiers to be *better than random*, as well as *uncorrelated*.

Bagging reduces variance by averaging over many trees. Random forests purposefully add "randomness" through the random features and decreases correlations of these trees. Think of random forest as an improved bagging method.

-----
# Boosting

Adaboost is a boosting implementation by Freund and Schapire that fit a sequence of weak learners. Each step improves upon the last one by overweighting misclassified samples and underweighting correct samples. The weak learners are also given weights based on their error rates.

There are many components in AdaBoost--Let's see how they function.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# Default decision tree classifier
#  Notice the learning rate
model_boost = AdaBoostClassifier()

SAMME works off of discrete misclassification errors, while SAMME.R uses the predicted probabilities to reweight the samples and base classifiers.

In [ ]:
model_boost.fit(X_train,y_train)

In [ ]:
model_boost.score(X_train,y_train)

Boosting tends to perform better when the base classifier has a high bias but low variance.

I.e. Random forest works well with **OVERFIT** models, while Boosting works well with **UNDERFIT** models.

# Exercise - Stumps vs. Dense Trees

Remember that in order for ensemble learners to work well, they require that the base classifiers are better than random guessing, and tend to get 'different questions wrong'. 

Boosting often does surprisingly well when using decision tree stumps (i.e. Decision Tree Classifiers with depth=1).

Create a plot showing in sample and out of sample performance for an adaboost ensemble classifier that uses base_estimator as Decision Tree Classifiers with max depth = 1 and 10, as you increase the number of estimators.

The x-axis should be n_estimators while y-axis is the score (or error = 1 - score). There should be two lines, one for stump and 1 for 10 node trees.

Warning: Make sure your code works correctly--this can take time! I encourage you to try printing out diagnostics too to help you see that it is working.

Also, just using stumps, observe the performance in and out of sample as you increase the n_estimators from 1 to 100. You may want to test various ranges, rather than every number between 1 and 100.

If you finish early, feel free to play around with the GBDT (Gradient Boosted Decision Tree), aka GBT (Gradient Boosted Tree), aka GBM (Gradient Boosted Model) within sklearn. [GBDT](http://scikit-learn.org/stable/modules/ensemble.html#gradient-tree-boosting)

Gradient boosting can be thought of as a generic form of boosting that can take in any differentiable loss function.